# Ayudantía 5

## Excepciones y Testing

**Raúl Álvarez y Octavio Vera**

El siguiente código está **muy** mal por al menos dos razones. **¿Por qué?**

In [1]:
try:
    # Do something
    raise TypeError
except:
    # Do something else
    pass

### Motivos

- Motivo 1:

Muchas veces utilizamos erróneamente las excepciones, y en vez de efectivamente manejar 
las situaciones adecuadamente, generamos un código "parche" que permite a nuestro programa 
funcionar de forma equivocada (no deseada) e igualmente funcionar. Esto permite como última
medida funcionar a tu código, pero sin cumplir la tarea que en verdad se desea realizar.

- Motivo 2:

Elevar un error al manejar otro es contraproducente. No sólo implica que se debe volver a manejar
el error que se crea a partir de ese punto, sino que genera ambiguedad con respecto al tipo
de error que realmente es. 

- Motivo 3:

Realizamos una acción, previo a levantar otro error, más allá de lo mencionado en el motivo 2,
esto puede complicar entender el estado previo a que se levante ese error, generando así una
cadena engorrosa de secuencias de ejecución y manejos de errores.


#### Entonces...

¿Cuándo se utiliza `raise`?

¿Cuándo se utiliza `try` / `except`?

## `raise`

In [2]:
class Fraccion:
    """ Implementamos la clase Fraccion que nos permite trabajar sobre fracciones
    enteras. Buscamos complementar el modulo "math" tal que sea posible representar
    las fracciones enteras de forma facil y comoda, para utilizarlo tanto matematicamente
    como visualmente una vez creadas las instancias.
    """

    def __init__(self, num, den):
        if not (isinstance(num, int) and isinstance(den, int)):
            raise TypeError("Al menos un argumento no es entero.")
        elif den == 0:
            raise ZeroDivisionError("El denominador no puede ser 0.")
        self.num = num
        self.den = den
        self.__simplificar()

    def __simplificar(self):
        i = 1
        while i <= self.num or i <= self.den:
            if self.num % i == 0 and self.den % i == 0:
                self.num = self.num // i
                self.den = self.den // i
                i = 1
            i += 1

    def __eq__(self, other):
        """Con este método mágico implementamos lo necesario para poder
        comparar dos fracciones con el operador ==.
        Fraccion(1, 2) == Fraccion(2, 3)
        Se revisa además que el argumento other sea efectivamente
        una Fraccion."""
        if isinstance(other, Fraccion):
            return self.num == other.num and self.den == other.den
        else:
            raise TypeError

    def __lt__(self, other):
        """Con este método se implementa el operador booleano <"""
        if isinstance(other, Fraccion):
            return self.num * other.den < other.num * self.den
        else:
            raise TypeError

    def __le__(self, other):
        """Con este método se implementa el operador booleano <=. A partir
        de los operadores booleanos implementados hasta ahora, Python
        puede deducir !=, >, >="""
        if isinstance(other, Fraccion):
            return self.num * other.den <= other.num * self.den
        else:
            raise TypeError

    def __add__(self, other):
        """Con este método mágico implementamos lo necesario para poder
        sumar dos fracciones con el operador +.
        Fraccion(1, 2) + Fraccion(2, 3)
        Se revisa además que el argumento other sea efectivamente
        una Fraccion."""
        if isinstance(other, Fraccion):
            num = self.num * other.den + self.den * other.num
            den = self.den * other.den
            return Fraccion(num, den)
        else:
            raise TypeError

    def __neg__(self):
        """Operador de la negación para poder usar (- Fraccion(1, 2))"""
        return Fraccion(-self.num, self.den)

    def __sub__(self, other):
        """Operador de la resta con el símbolo -"""
        if isinstance(other, Fraccion):
            return (Fraccion(self.num, self.den) +
                    - Fraccion(other.num, other.den))
        else:
            raise TypeError

    def __mul__(self, other):
        """Operador de la multiplicación *"""
        if isinstance(other, Fraccion):
            num = self.num + other.num
            den = self.den * other.den
            return Fraccion(num, den)
        else:
            raise TypeError

    def __truediv__(self, other):
        """Operador de la división /"""
        if isinstance(other, Fraccion):
            num = self.num - other.den
            den = self.den * other.num
            return Fraccion(num, den)
        else:
            raise TypeError

    def __str__(self):
        """Representación en string, legible para un usuario."""
        return "{}/{}".format(self.num, self.den)

    def __repr__(self):
        """Representación en string, legible para el programador."""
        return "Fraccion({}, {})".format(self.num, self.den)


Idea: Pedir que se cree/cambie una función con un custom exception.

## `try/except`

In [3]:
def crear_fraccion(num, den):
    try:
        # Intentamos ejecutar el código de este bloque.
        f = Fraccion(num, den)
        return f
    except (TypeError, ZeroDivisionError) as err:
        # En caso de que se levante una excepción de los tipos en la
        # tupla, ésta se guarda en la variable err y puede ser procesada.
        print(err)
        return None


def ejercicios():
    # definimos e instanciamos las fracciones #
    f1 = crear_fraccion(3, 8)
    f2 = crear_fraccion(1, 4)
    f3 = crear_fraccion(1, 2)
    f4 = crear_fraccion(5, 4)
    f5 = crear_fraccion("6", 0)

    try:
        ejercicio_1 = ((f1 + f2) / f3) == f4
    except TypeError:
        print("El ejercicio 1 presenta un error")
    else:
        print("Solucion 1: {}".format(ejercicio_1))
    finally:
        print("Esto se imprime siempre, haya o no resultado el ejercicio 1.")

    try:
        ejercicio_2 = (f5 + f3) * f2
    except TypeError:
        print("El ejercicio 2 presenta un error")
    else:
        print("Solucion 2: {}".format(ejercicio_2))
    finally:
        print("Esto se imprime siempre, haya o no resultado el ejercicio 2.")


ejercicios()

Al menos un argumento no es entero.
Solucion 1: False
Esto se imprime siempre, haya o no resultado el ejercicio 1.
El ejercicio 2 presenta un error
Esto se imprime siempre, haya o no resultado el ejercicio 2.


## Testing

Aquí testeamos los métodos `__add__`, `__sub__`, `__mul__`, ..., etc., de la clase Fracción de tal forma que podamos evaluar distintas situaciones ocurrentes en la generación de instancias,
comparación, y operaciones matemáticas realizadas.

In [4]:
import unittest


class FraccionTest(unittest.TestCase):

    def setUp(self):
        self.tuplas_fraccion = [(3,4),(20,80),(1,27)]
        self.tuplas_error = [(4,0),(1.4,2),("2","7")]
        self.trivial = Fraccion(1,4)

    def tearDown(self):
        # No es necesario realizar un tearDown de esta clase,        #
        # ya que testearla no modificara su comportamiento a futuro. #
        # Y no es necesario restablecer un estado previo al setUp.   #
        pass

    def test_init(self):
        for tupla in self.tuplas_fraccion:
            # se nos permite instanciar estos valores sin problema  #
            Fraccion(*tupla)

        for tupla in self.tuplas_error:
            # verificamos que efectivamente se levante las excepciones #
            if tupla[1] == 0:
                self.assertRaises(ZeroDivisionError, Fraccion, *tupla)
            else:
                self.assertRaises(TypeError, Fraccion, *tupla)

    def test_eq(self):
        for tupla in self.tuplas_fraccion:
            # sabemos que el segundo es el unico igual             #
            if tupla == self.tuplas_fraccion[1]:
                self.assertTrue(self.trivial == Fraccion(*tupla))
            else:
                self.assertFalse(self.trivial == Fraccion(*tupla))

        for tupla in self.tuplas_error:
            # verificamos que efecitvamente se realizen los errores #
            self.assertRaises(TypeError, self.trivial.__eq__, tupla)

    def test_add(self):
        resultado = [Fraccion(4,4), Fraccion(2,4), Fraccion(31,108)]
        i = 0
        for tupla in self.tuplas_fraccion:
            # sumamos y comparamos con su resultado deseado         #
            self.assertEqual(self.trivial + Fraccion(*tupla), resultado[i])
            i += 1

        for tupla in self.tuplas_error:
            # verificamos que efecitvamente se realizen los errores #
            self.assertRaises(TypeError, self.trivial.__add__, tupla)

    def test_sub(self):
        resultado = [Fraccion(-2,4), Fraccion(0,4), Fraccion(23,108)]
        i = 0
        for tupla in self.tuplas_fraccion:
            # restamos y comparamos con su resultado deseado        #
            self.assertEqual(self.trivial - Fraccion(*tupla), resultado[i])
            i += 1

        for tupla in self.tuplas_error:
            # verificamos que efecitvamente se realizen los errores #
            self.assertRaises(TypeError, self.trivial.__sub__, tupla)

    def test_mul(self):
        resultado = [Fraccion(3,16), Fraccion(1,16), Fraccion(1,108)]
        i = 0
        for tupla in self.tuplas_fraccion:
            # multiplicamos y comparamos con su resultado deseado   #
            self.assertEqual(self.trivial * Fraccion(*tupla), resultado[i])
            i += 1

        for tupla in self.tuplas_error:
            # verificamos que efecitvamente se realizen los errores #
            self.assertRaises(TypeError, self.trivial.__mul__, tupla)

    def test_true_div(self):
        resultado = [Fraccion(1,3), Fraccion(1,1), Fraccion(27,4)]
        i = 0
        for tupla in self.tuplas_fraccion:
            # dividimos y comparamos con su resultado deseado       #
            self.assertEqual(self.trivial / Fraccion(*tupla), resultado[i])
            i += 1

        for tupla in self.tuplas_error:
            # verificamos que efecitvamente se realizen los errores #
            self.assertRaises(TypeError, self.trivial.__mul__, tupla)

    def test_view(self):
        # verificamos el comportamiento de __str__ #
        self.assertEqual(self.trivial.__str__(),"1/4")

        # verificamos el comportamiento de __repr__#
        lista = []
        for tupla in self.tuplas_fraccion:
            lista.append(Fraccion(*tupla))
        self.assertEqual(lista.__repr__(),"[Fraccion(3, 4), Fraccion(1, 4), Fraccion(1, 27)]")

    @unittest.skip("No implementado")
    def test_no_implementado(self):
        pass

Ahora, corremos esta suite de tests. Cada método es un test y estos se ejecutan en orden alfabético.

In [5]:
# Esta línea solo es necesaria en Jupyter Notebook
suite = unittest.TestLoader().loadTestsFromTestCase(FraccionTest)
unittest.TextTestRunner().run(suite)
# En un programa normal, basta con correr
# unittest.main()

...Fs.F.
FAIL: test_mul (__main__.FraccionTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-3b58f4b0acc5>", line 70, in test_mul
    self.assertEqual(self.trivial * Fraccion(*tupla), resultado[i])
AssertionError: Fraccion(1, 4) != Fraccion(3, 16)

FAIL: test_true_div (__main__.FraccionTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-3b58f4b0acc5>", line 82, in test_true_div
    self.assertEqual(self.trivial / Fraccion(*tupla), resultado[i])
AssertionError: Fraccion(-1, 4) != Fraccion(1, 3)

----------------------------------------------------------------------
Ran 8 tests in 0.014s

FAILED (failures=2, skipped=1)


<unittest.runner.TextTestResult run=8 errors=0 failures=2>

De arriba podemos notar que fallaron dos tests: `test_mul` y `test_true_div`. A continuación, corregiremos estos métodos.

In [6]:
def __mul__(self, other):
        """Operador de la multiplicación *"""
        if isinstance(other, Fraccion):
            # Corregimos la multiplicación de numeradores
            num = self.num * other.num
            den = self.den * other.den
            return Fraccion(num, den)
        else:
            raise TypeError

In [7]:
def __truediv__(self, other):
    """Operador de la división /"""
    if isinstance(other, Fraccion):
        # Corregimos la multiplicación cruzada
        num = self.num * other.den
        den = self.den * other.num
        return Fraccion(num, den)
    else:
        raise TypeError


In [8]:
# Esto lo veremos más adelante en el curso,
# pero así podemos reemplazar los métodos por los
# nuevos recién definidos.
setattr(Fraccion, "__mul__", __mul__)
setattr(Fraccion, "__truediv__", __truediv__)

Corremos nuevamente los tests.

In [9]:
suite = unittest.TestLoader().loadTestsFromTestCase(FraccionTest)
unittest.TextTestRunner().run(suite)

....s...
----------------------------------------------------------------------
Ran 8 tests in 0.011s

OK (skipped=1)


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

Y podemos ver que ahora sí se logran pasar todos los tests.